# Štatistické testovanie hypotéz

URL https://github.com/FIIT-IAU/

**Chceme overiť, či má počet valcov motora vplyv na spotrebu.**

In [ ]:
import pandas as pd
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
import scipy.stats as stats
from sklearn import preprocessing

In [ ]:
cars = pd.read_csv('data/auto-mpg.data', 
                   delim_whitespace=True, 
                   names = ['mpg', 'cylinders', 'displacement','horsepower',
                            'weight', 'acceleration', 'model_year', 'origin', 'name'],
                   na_values='?')
cars.head()

In [ ]:
cars.cylinders.unique()

In [ ]:
cars.cylinders.value_counts()

In [ ]:
sns.boxplot(x='cylinders', y='mpg', data=cars)

Vidíme, že v datasete máme viacero typov motora (podľa počtu valcov). Už z vizualizácie pomocou boxplotov vidíme, že existuje vzťah medzi počtom valcov a spotrebou (vyjadrenou ako počet míľ, ktoré je auto schopné prejsť na jeden galón paliva, t. j. miles per gallon, `mpg`).

Máme viacero možností, ako otestovať typ tohto vzťahu:

* môžeme sa pozrieť, či je medzi týmito dvomi atribútmi korelácia
* môžeme skúsiť *nafitovať* (napr. lineárny) regresný model
* môžeme otestovať rozdiely medzi priemermi skupín podľa počtu valcov.

My sa pozrieme na tú poslednú možnosť. Otestujme či je **rozdiel medzi spotrebou 6 a 8-valcových motorov štatisticky signifikantný (a teda nie je napr. len daný náhodou alebo chybou)**.

Určímte si teda naše hypotézy nasledovne:

**$H_0$ (nulová hypotéza)**: Spotreba 6-valcových motorov **je** v priemere **rovnaká** ako spotreba 8-valcových

**$H_1 = H_A$ (alternatívna hypotéza)**: Spotreba 6-valcových motorov **je** v priemere **iná/väčšia/menšia** ako spotreba 8-valcových


In [ ]:
sns.boxplot(x='cylinders', y='mpg', data=cars[(cars.cylinders == 6) | (cars.cylinders == 8)])

- Nejaký rozdiel tam je, to vidíme aj na základe vizuálneho porovnania. Ak chceme overiť, či je signifikantný, potrebujeme použiť štatistický test.
- Máme dve skupiny, vťah medzi nimi je nezávislý (motor má vždy buď 6 alebo 8 valcov). Do úvahy teda pripadá *t-test* alebo *Mann-Whitenyho U test*. Volíme t-test, ak sú splnené jeho predpoklady (dáta pochádzajú z normálnych distribúcií a tie majú aj rovnaké (prípadne podobné) variancie.

## Overenie predpokladov

### Predpoklad normálnosti rozdelenia

Normálnosť rozdelenia si vieme overiť vizuálne pomocou histogramu, prípadne pomocou tzv. QQ-grafu.

In [ ]:
mpg6 = cars.loc[cars.cylinders == 6, 'mpg']

In [ ]:
mpg6.describe()

In [ ]:
# sns.distplot(mpg6)
sns.histplot(mpg6)

Vo vzorke sa nachádzajú vychýlené hodnoty. Najjednoduchšou metódou na identifikáciu vychýlených hodnôt (outlierov) je označiť za vychýlenú hodnotu pozorovanie, ktoré sa odlišuje o viac ako 1,5 násobok medzikvartilového rozdelenia od horného, resp. dolného kvartilu.

In [ ]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
mpg6_out = identify_outliers(mpg6)
mpg6_out

In [ ]:
mpg6 = mpg6.drop(mpg6_out.index)

In [ ]:
# sns.distplot(mpg6)
sns.histplot(mpg6)

In [ ]:
mpg8 = cars.loc[cars.cylinders == 8, 'mpg']

In [ ]:
mpg8.describe()

In [ ]:
mpg8_out = identify_outliers(mpg8)
mpg8_out

In [ ]:
mpg8 = mpg8.drop(mpg8_out.index)

In [ ]:
# sns.distplot(mpg8)
sns.histplot(mpg8)

In [ ]:
_ = sm.ProbPlot(mpg6, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(mpg8, fit=True).qqplot(line='45')

QQ-plot je vizuálna metóda na určenie, či dve dátove sady pochádzajú z rovnakého rozdelenia. Najčastejšie sa porovnáva rozdelenie vzorky s teoretickým normálnym rozdelením. Bod na grafe zobrazuje hodnotu kvantilu v prvom a druhom porovnávanom datasete.

#### Na aké otazky vie QQ-plot odpovedať?

* Pochadzajú dve skupiny pozorovaní z rovnakého rozdelenia?
* Pochádza pozorovaná vzorka z testovaného teoretického rozdelenia (napr. normálneho)?
* Majú rozdelenia podobné vlastnosti asymetrie (skewness) a špicatosti (kurtosis)?

## Shapiro-Wilkov test normálnosti

Na overenie normálnosti vieme použiť aj **Shapiro-Wilkov test**, ktorý testuje nulovú hypotézu, že dáta pochádzajú z normálneho rozdelenia. Ak je $p < 0,05$, nulovú hypotézu $H_0$ zamietame a dáta pravdepodobne pochádzajú z iného ako normálneho rozdelenia. Ak je $p > 0,05$, nulovú hypotézu $H_0$ nezamietame, teda na základe dát nemôžeme prehlásiť, že by dáta pochádzali z iného, ako normálneho rozdelenia.

In [ ]:
stats.shapiro(mpg6)

In [ ]:
stats.shapiro(mpg8)

Na základe výsledkov testu sa zdá, že vzorka aút so 6-valcovými motormi pochádza z normálneho rozdelenia, vzorka s 8-valcovými motormi nie. Mali by sme teda použiť neparametrickú verziu t-testu, t. j. **Mann-Whitneyho U-test** (hoci t-test je nad istý počet vzoriek relatívne robustný na mierne odchýlky od predpokladu normálnosti).

## Podobnosť variancie

Druhým predpokladom použitia t-testu je rovnosť variancií (hoci existuje variant t-testu, ktorý vie pracovať aj s dátami s nerovnakou varianciou). Hoci sme neoverili predpoklad normálnosti rozdelení, pozrime sa na ich variancie. 

Na otestovanie podobnosti variancií sa používa **Levenov test**. Testuje nulovú hypotézu $H_0$, že všetky vstupné vzorky pochádzajú z rozdelení s rovnakými varianciami. Ak $H_0$ nezamietame ($p > 0,05$), znamená to, že na základe dát nemôžeme prehlásiť, že by vzorky pochádzali z distribúcií s rôznymi varianciami.

In [ ]:
stats.levene(mpg6, mpg8)

Na základe výsledku testu sa zdá, že vzorky pochádzajú z rozdelení s rovnakou varianciou.

## Studentov t-test vs. Mann-Whiteneyho U-test

Keďže neboli splnené predpoklady t-testu, mali by sme použiť jeho neparametrickú verziu. Ak by boli splnené, použili by sme funkciu `scipy.stats.ttest_ind`.

In [ ]:
stats.mannwhitneyu(mpg6, mpg8)

Keďže $p < 0,001$, pravdepodobnosť chyby 1. rádu (že $H_0$ je pravdivá a my ju zamietame) je menej ako 1 promile. Našu nulovú hypotézu $H_0$ teda zamietame v prospech alternatívnej hypotézy $H_A$. Rozdiel v spotrebe medzi 6 a 8-valcovými motormi je štatisticky signifikantný.

Môžeme si vizualizovať rozdiel medzi dvomi priemermi - často sa zobrazujú pomocou stĺpcových diagramov spolu s *intervalmi spoľahlivosti*, ktoré nám hovoria, že s N% pravdepodobnosťou (najčastejšie sa používa 95) sa skutočná hodnota priemeru bude nachádzať niekde v danom intervale.

In [ ]:
sms.DescrStatsW(mpg6).tconfint_mean()

In [ ]:
sms.DescrStatsW(mpg8).tconfint_mean()

In [ ]:
sns.barplot(x='cylinders', y='mpg', data=cars[(cars.cylinders == 8) | (cars.cylinders == 6)], 
            capsize=0.1, errwidth=2, palette=sns.color_palette("Blues"))